# Optimisation solutions
The optimum solution for each week differs. To improve converge quality and speed, the optimum for each week shall be calculated and stored

In [ ]:
# imports
import json
import os
import time
import warnings
from cProfile import Profile
from pstats import SortKey, Stats

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm

from src.grid import Grid
from src.generators import (
    SolarGenerator,
    WindGenerator,
    NuclearGenerator,
    CoalGenerator,
    GasGenerator,
    BatteryGenerator,
)
from src.utils import WEEK_MAP

In [ ]:
profile = Profile()
profile.enable()

# initialise dictionary of optimum initialisations
opt_init_weekly = {}

# loop over each week
for week in tqdm(WEEK_MAP["week"]):
    print(week)
    # initialise grid for current week
    grid = Grid(
        generators={
            "solar": SolarGenerator,
            "wind": WindGenerator,
            "nuclear": NuclearGenerator,
            "battery": BatteryGenerator,
            "gas": GasGenerator,
            "coal": CoalGenerator,
        },
        week=week,
    )

    # save successul optimisations and print unsuccessful ones
    if grid.optimum is not None:
        opt_init_weekly[int(week)] = {
            **{
                k: v / grid.demand.max()
                for k, v in grid.optimum["installed_capacity"].items()
            },
            **{"score": grid.optimum["score"]},
        }
    else:
        warnings.warn(f"No optimum found")

profile.disable()

In [ ]:
# display profiler information on most time consuming functions
Stats(profile).sort_stats(SortKey.TIME).print_stats(0.03)

In [ ]:
# display profiler information on most time consuming functions/sub functions
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(0.03)

In [ ]:
for week in tqdm(WEEK_MAP["week"]):
    assert week in opt_init_weekly

In [ ]:
# save file
with open(os.path.join(os.getcwd(), "weekly_optimum.json"), "w") as f:
    f.write(json.dumps(opt_init_weekly))

In [ ]:
# plot results
df = pd.DataFrame(opt_init_weekly).T
fig, ax_all = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))
for data, ax_row in zip(
    [df.drop(columns="score"), df[["score"]]],
    [[ax_all[0, 0], ax_all[0, 1]], [ax_all[1, 0], ax_all[1, 1]]],
):
    for ax in ax_row:
        sns.boxplot(data=data, orient="h", ax=ax)
        ax.grid()
ax_all[0, 0].set_xlim(left=0)
ax_all[1, 0].set_xlim(left=0)
ax_all[0, 1].set_xlim(left=0, right=3)
ax_all[1, 1].set_xlim(left=0, right=100)
fig.tight_layout()